In [1]:
import sqlite3
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import re
import os
import pyspark

In [2]:
today = pd.Timestamp("today").strftime("%Y%m%d_%H%M%S")
save_location = "/home/pmata/mepram_data/"
db_filename = "db_mepram_sepsis_vf.sqlite3"

In [3]:
def codes_to_names(tab_to_process, codes_tab, shortname):
    errors = []
    codes_group = codes_tab.filter(F.col("form") == shortname)
    
    try:
        codes_group = codes_group.withColumn("value", F.col("value").cast("long"))
    except Exception:
        pass
    
    codes_group = codes_group.filter(F.col("value").isNotNull())
    
    if codes_group.count() == 0:
        print("NO VARS TO RENAME IN:", shortname)
        return tab_to_process, errors
    
    variables = [row.variable for row in codes_group.select("variable").distinct().collect()]
    
    for varname in variables:
        if varname == "fenotipo_resist_colo":
            varname = "feno_resist_colo"
        
        vartab = codes_group.filter(F.col("variable") == varname)
        
        mapping_rows = vartab.select("value", "name").collect()
        codes2names_dict = {row["value"]: row["name"] for row in mapping_rows}
        
        print(f"RENAMING {varname} for {codes2names_dict}:")
        
        try:
            mapping_expr = F.create_map([F.lit(x) for kv in codes2names_dict.items() for x in kv])
            
            tab_to_process = tab_to_process.withColumn(
                varname,
                mapping_expr.getItem(F.col(varname).cast("string")).cast("string")
            )
        
        except Exception as e:
            errors.append(
                f"Could not rename {varname}. Columns found: {tab_to_process.columns}. Error: {e}"
            )
            continue
    
    return tab_to_process, errors

"""db_path = os.path.join(save_location, "db_mepram_sepsis_vf.sqlite3")
con = sqlite3.connect(db_path)
tablenames = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type = 'table'", con
)

tablenames = sorted([x for x in tablenames][:-1])
tableshort = sorted([x.replace("tbl_", "") for x in tablenames])
print(tableshort)
codes_tab = pd.read_sql_query("SELECT * from tbl_codes2names", con)
codes_tab = codes_tab.drop("recode")
all_tabs = {}
errors = {}
for tabname, shortname in zip(tablenames, tableshort):
    print("tabname:", tabname, "/// shortname:", shortname)
    tab_to_process = pd.read_sql_query(
        f"SELECT * from {tabname}", con, infer_schema_length=None
    )
    all_tabs[tabname], errors[tabname] = codes_to_names(
        tab_to_process, codes_tab, shortname
    )"""

'db_path = os.path.join(save_location, "db_mepram_sepsis_vf.sqlite3")\ncon = sqlite3.connect(db_path)\ntablenames = pd.read_sql_query(\n    "SELECT name FROM sqlite_master WHERE type = \'table\'", con\n)\n\ntablenames = sorted([x for x in tablenames][:-1])\ntableshort = sorted([x.replace("tbl_", "") for x in tablenames])\nprint(tableshort)\ncodes_tab = pd.read_sql_query("SELECT * from tbl_codes2names", con)\ncodes_tab = codes_tab.drop("recode")\nall_tabs = {}\nerrors = {}\nfor tabname, shortname in zip(tablenames, tableshort):\n    print("tabname:", tabname, "/// shortname:", shortname)\n    tab_to_process = pd.read_sql_query(\n        f"SELECT * from {tabname}", con, infer_schema_length=None\n    )\n    all_tabs[tabname], errors[tabname] = codes_to_names(\n        tab_to_process, codes_tab, shortname\n    )'

# DB Import


In [4]:
import pandas as pd
import numpy as np
import sqlite3
import re
import os
con = sqlite3.connect(os.path.join(save_location, db_filename))

cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

"""with open(os.path.join(save_location, "tbl_personid2center.sql"), "r") as f:
    sql_script = f.read()
    cursor.executescript(sql_script)

with open(os.path.join(save_location, "tbl_microorganismos.sql"), "r") as f:
    sql_script = f.read()
    cursor.executescript(sql_script)"""

tables = [row[0] for row in cursor.fetchall()]

print(tables)

dataframes = {}
for table in tables:
    print(f"Cargando la tabla: {table}")
    dataframes[table] = pd.read_sql_query(f"SELECT * FROM {table}", con)
con.close()



['tbl_master', 'tbl_paciente', 'tbl_comorbilidad', 'tbl_factores_riesgo_bmr', 'tbl_sintomas', 'tbl_signos', 'tbl_sepsis', 'tbl_infecciones_previas', 'tbl_colonizaciones_previas', 'tbl_tratamiento_antibiotico_previo', 'tbl_tratamiento_empirico', 'tbl_hemocultivo_de_urgencias', 'tbl_otros_cultivos_en_urgencias', 'tbl_codes2names', 'tbl_personid2center', 'tbl_microorganismos']
Cargando la tabla: tbl_master
Cargando la tabla: tbl_paciente
Cargando la tabla: tbl_comorbilidad
Cargando la tabla: tbl_factores_riesgo_bmr
Cargando la tabla: tbl_sintomas
Cargando la tabla: tbl_signos
Cargando la tabla: tbl_sepsis
Cargando la tabla: tbl_infecciones_previas
Cargando la tabla: tbl_colonizaciones_previas
Cargando la tabla: tbl_tratamiento_antibiotico_previo
Cargando la tabla: tbl_tratamiento_empirico
Cargando la tabla: tbl_hemocultivo_de_urgencias
Cargando la tabla: tbl_otros_cultivos_en_urgencias
Cargando la tabla: tbl_codes2names
Cargando la tabla: tbl_personid2center
Cargando la tabla: tbl_microor

In [5]:
import re
for name, table in dataframes.items():
    print(name.replace("tbl", "formulario"))
    print("Columnas: ", re.sub(r"[\[\]]", "", str([x for x  in table.columns])))


formulario_master
Columnas:  'person_id', 'fecha_ingreso_urgencias', 'valido', 'record_id'
formulario_paciente
Columnas:  'person_id', 'fecha_ingreso_urgencias', 'fecha_nacimiento', 'edad', 'sexo', 'codigo_postal', 'mujer_gestante', 'mayor_65', 'paciente_residencia'
formulario_comorbilidad
Columnas:  'person_id', 'fecha_ingreso_urgencias', 'infarto', 'insuficiencia_cardiaca', 'evp', 'e_cerebrovascular', 'demencia', 'e_pulmonar_cronica', 'ulcera_peptica', 'colagenopatia', 'hemiplejia', 'erc', 'neoplasia', 'tipo_cancer', 'linfoma', 'leucemia', 'sida', 'hepatopatia', 'tipo_hepatopatia', 'diabetes', 'indice_de_charlson', 'inmunosupresion', 'causa_inmunosupresion', 'situacion_funcional_basal'
formulario_factores_riesgo_bmr
Columnas:  'person_id', 'fecha_ingreso_urgencias', 'hospit_ano_previo', 'hospit_mes_previo', 'hospit_ano_previo_uci', 'cirugia_previa_sin_implant', 'cirugia_previa_con_implant', 'asistencia_sanitaria_prev', 'hemodialisis_permanente', 'dialisis_peritoneal', 'cateter_venoso

#### Function to replace outliers by NaN using IQR and recode quantitative variables into ranges from 0 to 3.  

In [6]:
def process_variables(df, variables):
    """
    This function replaces outliers with NaN using IQR and recodes quantitative variables into ranges 0-3.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    variables (list): List of column names to process.

    Returns:
    pd.DataFrame: The DataFrame with outliers replaced by NaN and new recoded columns.
    """

    df_processed = df.copy()
    
    for var in variables:
        if var not in df.columns:
            print(f"Variable '{var}' not found in the DataFrame. It will be skipped.")
            continue

        Q1 = df_processed[var].quantile(0.25)
        Q3 = df_processed[var].quantile(0.75)
        IQR = Q3 - Q1

        lower_limit = Q1 - 3 * IQR
        upper_limit = Q3 + 3 * IQR

        outliers_mask = (df_processed[var] < lower_limit) | (df_processed[var] > upper_limit)
        outliers_count = outliers_mask.sum()
        if outliers_count > 0:
            print(f"{outliers_count} outliers replaced with NaN in the variable '{var}'.")
            print(f"{df_processed.loc[outliers_mask, var]}")            

        df_processed.loc[outliers_mask, var] = np.nan

        new_column = f"{var}_recoded"

        df_processed[new_column] = pd.qcut(df_processed[var], 
                                           q=4, 
                                           labels=[0, 1, 2, 3], 
                                           duplicates='drop')

        df_processed[new_column] = df_processed[new_column].astype('Int64') 
        
        print(f"Variable '{var}' recoded in the column '{new_column}'.")
    
    return df_processed

# tbl_paciente

In [7]:
df_pacientes = dataframes["tbl_paciente"].merge(dataframes["tbl_personid2center"], how="left")

df_pacientes["inf_previa_sino"] = np.where(np.isin(df_pacientes["person_id"].values, dataframes['tbl_infecciones_previas']["person_id"].values) == True, 1, 0)

bmr_previa = dataframes['tbl_infecciones_previas'].groupby("person_id")["bmr_infec_previa"].apply(lambda x: (x > 0).any())

df_pacientes = df_pacientes.merge(bmr_previa, on="person_id", how="left").fillna(False)
df_pacientes["bmr_infec_previa"] = np.where(df_pacientes["bmr_infec_previa"], 1, 0)


/tmp/ipykernel_4137759/3359684473.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_pacientes = df_pacientes.merge(bmr_previa, on="person_id", how="left").fillna(False)


# tbl_comorbilidad

In [8]:
tbl_comorbilidad = dataframes['tbl_comorbilidad']
tbl_comorbilidad = pd.get_dummies(tbl_comorbilidad, columns=["tipo_cancer","tipo_hepatopatia"])
print(tbl_comorbilidad)

      person_id fecha_ingreso_urgencias  infarto  insuficiencia_cardiaca  evp  \
0             1              2021-04-22        0                       0    0   
1             2              2021-05-07        0                       0    0   
2             3              2021-06-04        0                       0    0   
3             4              2021-06-22        0                       0    0   
4             5              2021-03-24        0                       0    0   
...         ...                     ...      ...                     ...  ...   
3908       3909              2023-01-18        0                       1    0   
3909       3910              2023-04-23        0                       0    0   
3910       3911              2023-04-10        0                       0    0   
3911       3912              2023-06-19        0                       0    0   
3912       3913              2023-06-26        0                       0    0   

      e_cerebrovascular  de

# tbl_factores_riesgo_bmr

In [9]:
tbl_factores_riesgo_bmr = dataframes['tbl_factores_riesgo_bmr']
print(tbl_factores_riesgo_bmr)

      person_id fecha_ingreso_urgencias  hospit_ano_previo  hospit_mes_previo  \
0             1              2021-04-22                  1                  1   
1             2              2021-05-07                  1                  1   
2             3              2021-06-04                  1                  1   
3             4              2021-06-22                  1                  1   
4             5              2021-03-24                  0                  0   
...         ...                     ...                ...                ...   
3908       3909              2023-01-18                  0                  0   
3909       3910              2023-04-23                  0                  0   
3910       3911              2023-04-10                  1                  1   
3911       3912              2023-06-19                  0                  0   
3912       3913              2023-06-26                  0                  0   

      hospit_ano_previo_uci

# tbl_sintomas

1. **One-Hot Encoding con Duración**:
   - Se crea una tabla pivote donde cada síntoma es una columna, y el valor corresponde a la suma de los días que el síntoma estuvo presente durante el ingreso.
   - Los valores nulos se reemplazan por 0, indicando ausencia del síntoma.
   - Finalmente los sintomas se recategorizan en: 
     - 0: Ausencia 
     - 1: Agudo (1-7 días)
     - 2: Larga duración (>7 días)

2. **Variables de Presencia/Ausencia**:
   - A partir de la tabla de duración, se generan variables binarias que indican si el síntoma estuvo presente (1) o no (0).


In [10]:
tbl_sintomas = dataframes['tbl_sintomas']
tbl_sintomas["sintoma"] = tbl_sintomas["sintoma"].astype(str)
tbl_sintomas["sintoma"] = "sintoma_" + tbl_sintomas["sintoma"]
tbl_sintomas_pivoted = tbl_sintomas.pivot_table(
    index=["person_id", "fecha_ingreso_urgencias"],  
    columns="sintoma",
    values="duracion_sintoma",
    aggfunc="sum",
    fill_value=0).reset_index()

presence_absence= tbl_sintomas_pivoted.copy()
presence_absence.iloc[:, 2:] = (presence_absence.iloc[:, 2:] > 0).astype(int)

recategorized= tbl_sintomas_pivoted.copy()
recategorized.iloc[:, 2:] = recategorized.iloc[:, 2:].applymap(
    lambda x:0 if x== 0 else (1 if x <= 7 else 2)
)

tbl_sintomas_complete = presence_absence.merge(
    recategorized, on= ['person_id', 'fecha_ingreso_urgencias'], suffixes= ("", "_categorico")
)

print(tbl_sintomas_complete)

sintoma  person_id fecha_ingreso_urgencias  sintoma_1.0  sintoma_10.0  \
0                1              2021-04-22          1.0           0.0   
1                2              2021-05-07          1.0           0.0   
2                3              2021-06-04          1.0           0.0   
3                4              2021-06-22          1.0           0.0   
4                5              2021-03-24          1.0           0.0   
...            ...                     ...          ...           ...   
3908          3909              2023-01-18          1.0           1.0   
3909          3910              2023-04-23          1.0           0.0   
3910          3911              2023-04-10          1.0           0.0   
3911          3912              2023-06-19          1.0           0.0   
3912          3913              2023-06-26          1.0           0.0   

sintoma  sintoma_11.0  sintoma_12.0  sintoma_13.0  sintoma_14.0  sintoma_15.0  \
0                 0.0           0.0       

/tmp/ipykernel_4137759/2736102941.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  recategorized.iloc[:, 2:] = recategorized.iloc[:, 2:].applymap(


# tbl_signos

In [11]:
tbl_signos = dataframes['tbl_signos']

for col in tbl_signos.columns[2:]:
    tbl_signos[col] = tbl_signos[col].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x)))
    tbl_signos[col] = tbl_signos[col].apply(lambda x: float(x[0]) if x else None)
    
tbl_signos['hipotermia_hipertermia'] = np.where(tbl_signos['temperatura'] >= 38, 2,
                                np.where(tbl_signos['temperatura'] >= 36, 0, 1))
tbl_signos['hipotermia_hipertermia'] = tbl_signos['hipotermia_hipertermia'].where(tbl_signos['temperatura'].notna())

#tbl_signos = tbl_signos.drop(columns=['hipotermia_hipertermia'])
tbl_signos['hipotension'] = np.where(tbl_signos['tension_arterial'].isna(), tbl_signos['hipotension'],
                                     np.where(tbl_signos['tension_arterial'] <= 100, 1, 0))
tbl_signos['taquipnea'] = np.where(tbl_signos['frec_respiratoria'].isna(), tbl_signos['taquipnea'],
                                   np.where(tbl_signos['frec_respiratoria'] > 20, 1, 0))
tbl_signos['taquicardia'] = np.where(tbl_signos['frec_cardiaca'].isna(), tbl_signos['taquicardia'],
                                     np.where(tbl_signos['frec_cardiaca'] >90, 1, 0))
tbl_signos['hipoxemia'] = np.where(tbl_signos['saturacion_o2'].isna(), tbl_signos['hipoxemia'],
                                     np.where(tbl_signos['saturacion_o2'] >90, 0, 1))
print(tbl_signos)

      person_id fecha_ingreso_urgencias  temperatura  hipotermia_hipertermia  \
0             1              2021-04-22         37.2                       0   
1             2              2021-05-07         37.0                       0   
2             3              2021-06-04         37.9                       0   
3             4              2021-06-22         37.8                       0   
4             5              2021-03-24         39.0                       2   
...         ...                     ...          ...                     ...   
3908       3909              2023-01-18         38.0                       2   
3909       3910              2023-04-23         37.6                       0   
3910       3911              2023-04-10         37.8                       0   
3911       3912              2023-06-19         38.0                       2   
3912       3913              2023-06-26         37.6                       0   

      frec_cardiaca  taquicardia  frec_

Remove outliers and recode

In [12]:
vairables= ['temperatura', 'frec_respiratoria', 'frec_cardiaca', 'tension_arterial','saturacion_o2']
tbl_signos = process_variables(tbl_signos, vairables)

Variable 'temperatura' recoded in the column 'temperatura_recoded'.
9 outliers replaced with NaN in the variable 'frec_respiratoria'.
293     46.0
553     50.0
2325    48.0
2400    50.0
2442    52.0
2481    52.0
3433    47.0
3461    51.0
3477    52.0
Name: frec_respiratoria, dtype: float64
Variable 'frec_respiratoria' recoded in the column 'frec_respiratoria_recoded'.
Variable 'frec_cardiaca' recoded in the column 'frec_cardiaca_recoded'.
Variable 'tension_arterial' recoded in the column 'tension_arterial_recoded'.
70 outliers replaced with NaN in the variable 'saturacion_o2'.
385     75.0
401     80.0
617     78.0
620     75.0
638     80.0
        ... 
3652    70.0
3660    70.0
3670    75.0
3810    70.0
3818    70.0
Name: saturacion_o2, Length: 70, dtype: float64
Variable 'saturacion_o2' recoded in the column 'saturacion_o2_recoded'.


# tbl_sepsis

In [13]:
tbl_sepsis = dataframes['tbl_sepsis']
tbl_sepsis['lactato_serico'] = np.where(tbl_sepsis['lactato_serico'] == '<= 2 millimole per liter', 0, 1)
for col in tbl_sepsis.columns[2:]:
    tbl_sepsis[col] = tbl_sepsis[col].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x)))
    tbl_sepsis[col] = tbl_sepsis[col].apply(lambda x: float(x[0]) if len(x) > 0 else None)
print(tbl_sepsis)

# Remove outliers and recode

variables =['proteina_c_reactiva']
tbl_sepsis = process_variables(tbl_sepsis, variables)

      person_id fecha_ingreso_urgencias  foco  sepsis  shock_septico  sofa  \
0             1              2021-04-22   1.0     1.0            0.0   6.0   
1             2              2021-05-07   8.0     0.0            0.0   4.0   
2             3              2021-06-04  12.0     0.0            0.0   5.0   
3             4              2021-06-22  12.0     1.0            0.0   6.0   
4             5              2021-03-24   1.0     0.0            0.0   4.0   
...         ...                     ...   ...     ...            ...   ...   
3908       3909              2023-01-18   4.0     0.0            0.0   1.0   
3909       3910              2023-04-23   3.0     1.0            0.0   2.0   
3910       3911              2023-04-10   4.0     1.0            0.0   3.0   
3911       3912              2023-06-19   2.0     1.0            0.0   2.0   
3912       3913              2023-06-26   4.0     0.0            0.0   1.0   

      respiracion  snc_glasgow  cardiovascular  bilirrubina  pl

# tbl_infecciones_previas

### Recodificación de microorganismos y dummy vars: 
Categorias restantes: "ecoli", "virus", "fungi", "Staphylococcus aureus", "Pseudomonas aeruginosa", "Klebsiella pneumoniae", "Streptococcus pneumoniae", "Enterococcus", "Enterobacteria", "other bacteria"

### Calculo de eventos y tiempo medio entre eventos/paciente

Se calcula el número de eventos por paciente (número de infecciones previas)
Se calcula el tiempo entre eventos y en el caso de pacientes con >2 eventos, se calcula el tiempo medio. 

In [ ]:
# Load table with classification group for each organism in column label
organism_classification = dataframes["tbl_microorganismos"].copy()

# Rename default values for final classification, use _ to separate target variables
label_map = {
    "NOEB": "_Other bacteria",
    "VIRUS": "_Virus",
    "FUNGUS": "_Fungi",
    "OEB": "_Enterobacteria",
    "ECOLI": "Escherichia coli",
    "SA": "Staphylococcus aureus",
    "PSA": "Pseudomonas aeruginosa",
    "KP": "Klebsiella pneumoniae",
    "SP": "Streptococcus pneumoniae",
    "EC": "Enterococcus"
}
organism_classification["label"] = organism_classification["label"].map(label_map)
# organism_classification[organism_classification.duplicated(subset="snomed_code", keep=False)].sort_values(by=["snomed_code", "label"])
# There are snomed_codes with 2 labels, so always keep the most informative
organism_classification = organism_classification.sort_values(by=["snomed_code", "label"]).drop_duplicates(subset="snomed_code", keep="first")
# Create a dictionary with {organism-snomed-code (value) : classification group (label)}
organism_codes_map = dict(zip(organism_classification["snomed_code"], organism_classification["label"]))
print(organism_codes_map)

{'10021006': '_Enterobacteria', '10049004': '_Other bacteria', '10091000087105': '_Other bacteria', '10151000087104': '_Fungi', '1017006': '_Other bacteria', '10171000087105': '_Fungi', '10210008': '_Fungi', '10262005': 'Enterococcus', '10334001': '_Enterobacteria', '103427005': '_Other bacteria', '103428000': '_Other bacteria', '103429008': 'Escherichia coli', '103434007': '_Enterobacteria', '103435008': '_Enterobacteria', '103436009': 'Enterococcus', '103437000': 'Enterococcus', '103438005': 'Enterococcus', '103447002': '_Other bacteria', '103448007': '_Other bacteria', '103449004': '_Other bacteria', '103450004': '_Other bacteria', '103452007': '_Other bacteria', '103453002': '_Other bacteria', '103454008': '_Other bacteria', '103455009': '_Other bacteria', '103459003': '_Other bacteria', '103460008': '_Other bacteria', '103461007': '_Other bacteria', '103462000': '_Other bacteria', '103474001': '_Other bacteria', '103475000': '_Other bacteria', '103476004': '_Other bacteria', '1034

In [15]:
tbl_infecciones_previas_mer = dataframes['tbl_infecciones_previas'].copy()


# Pasamos a dummy los microorganismos y rellenamos la información con el sumatorio de cada dummy. De este modo no perdemos información.
# Finalmente lo almacenamos en tbl_infecciones_previas_microorganismo
tbl_infecciones_previas_mer['grupo_microorganismo'] = tbl_infecciones_previas_mer['microorganism_infec_prev'].astype(str).map(organism_codes_map)
tbl_infecciones_previas_mer = tbl_infecciones_previas_mer.drop(columns=['microorganism_infec_prev', 'bmr_infec_previa', 'feno_resist_infec_prev'])
tbl_infecciones_previas_mer["dummy"] = 1
#tbl_infecciones_previas_mer = pd.get_dummies(tbl_infecciones_previas_mer, columns=['feno_resist_infec_prev'])
tbl_infecciones_previas_microorganismos = tbl_infecciones_previas_mer.pivot_table(
    index=["person_id", "fecha_ingreso_urgencias"],  
    columns="grupo_microorganismo",
    values="dummy",
    aggfunc="sum",
    fill_value=0).reset_index()

# Hemocultivo de urgencias (target variable)

In [17]:
import copy
hemo_urg = copy.deepcopy(dataframes['tbl_hemocultivo_de_urgencias'])
hemo_urg["microorganismo"] = hemo_urg["microorganismo"].fillna("0").astype(int).astype(str).map(lambda x: organism_codes_map.get(x, "NEGATIVE"))
hemo_urg["microorganismo"].value_counts()

microorganismo
NEGATIVE                    2074
Escherichia coli            1420
Klebsiella pneumoniae        504
_Enterobacteria              293
_Other bacteria              273
Staphylococcus aureus        199
Pseudomonas aeruginosa       111
Streptococcus pneumoniae     106
Enterococcus                  56
_Fungi                         8
Name: count, dtype: int64

In [18]:
dataframes['tbl_hemocultivo_de_urgencias']

,person_id,fecha_ingreso_urgencias,id_hemocultivo,fecha_hemocultivo,hemo_positivo_si_no,microorganismo,bmr_etiologia,fenotipo_resistencia
0,1,2021-04-22,621-2,2021-04-22,0.0,NaN,NaN,NaN
1,2,2021-05-07,212164727,2021-05-07,1.0,90272000.0,0.0,NaN
2,3,2021-06-04,621-4,2021-06-04,0.0,NaN,NaN,NaN
3,4,2021-06-22,621-5,2021-06-22,0.0,NaN,NaN,NaN
4,5,2021-03-24,621-10,2021-03-24,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5039,3912,2023-06-19,637-400,2023-06-19,1.0,112283007.0,1.0,6.0
5040,3912,2023-06-19,637-400,2023-06-19,1.0,112283007.0,1.0,7.0
5041,3912,2023-06-19,637-400,2023-06-19,1.0,112283007.0,1.0,8.0
5042,3912,2023-06-19,637-400,2023-06-19,1.0,112283007.0,1.0,9.0


In [19]:
hemo_urg_pivoted = hemo_urg.drop(["bmr_etiologia", "fenotipo_resistencia"], axis=1).pivot_table(
    index=["person_id", "fecha_ingreso_urgencias"],  
    columns="microorganismo",
    values="hemo_positivo_si_no",
    aggfunc="sum",
    fill_value=0).reset_index()
hemo_urg_pivoted

microorganismo,person_id,fecha_ingreso_urgencias,Enterococcus,Escherichia coli,Klebsiella pneumoniae,NEGATIVE,Pseudomonas aeruginosa,Staphylococcus aureus,Streptococcus pneumoniae,_Enterobacteria,_Fungi,_Other bacteria
0,1,2021-04-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2021-05-07,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2021-06-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2021-06-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,2021-03-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3908,3909,2023-01-18,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3909,3910,2023-04-23,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3910,3911,2023-04-10,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3911,3912,2023-06-19,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:

def get_bacteria(row):
    return ", ".join([col for col in row.index if row[col] == 1])

hemo_urg_pivoted["resultado_hemo"] = hemo_urg_pivoted.drop(columns=["person_id", "fecha_ingreso_urgencias"]).apply(get_bacteria, axis=1)
hemo_urg_pivoted["resultado_hemo"] = hemo_urg_pivoted["resultado_hemo"].replace("", "NEGATIVE").apply(lambda x: tuple(x.split(", ")))

# tbl_colonizaciones_previas

In [22]:
colo_prev = copy.deepcopy(dataframes["tbl_colonizaciones_previas"])
colo_prev["microorganism_colonizador"] = colo_prev["microorganism_colonizador"].astype(str).map(organism_codes_map)

In [23]:
colo_prev["dummy"] = 1
colo_prev_pivoted = colo_prev.pivot_table(
    index=["person_id", "fecha_ingreso_urgencias"],  
    columns="microorganism_colonizador",
    values="dummy",
    aggfunc="sum",
    fill_value=0).reset_index()
colo_prev_pivoted = colo_prev_pivoted.drop(columns=["fecha_ingreso_urgencias"])
colo_prev_pivoted.columns = ["colo_" + str(col) if col not in ["person_id", "fecha_ingreso_urgencias"] else col for col in colo_prev_pivoted.columns]

In [24]:
# Calculo del número de eventos por paciente y el tiempo medio entre cada visita. 

tbl_visitas_ip = tbl_infecciones_previas_mer.copy()
fechas_invalidas = tbl_visitas_ip[~tbl_visitas_ip['fecha_infeccion'].str.match(r'\d{4}-\d{2}-\d{2}')]
tbl_visitas_ip.loc[tbl_visitas_ip['fecha_infeccion'] == '323-05-01', 'fecha_infeccion'] = '2023-05-01'
tbl_visitas_ip['fecha_infeccion'] = pd.to_datetime(tbl_visitas_ip['fecha_infeccion'])
tbl_visitas_ip = tbl_visitas_ip[['person_id', 'fecha_ingreso_urgencias', 'fecha_infeccion']]
num_visitas = tbl_visitas_ip.groupby('person_id')['fecha_infeccion'].nunique().reset_index(name='numero_visitas')
print(num_visitas)

      person_id  numero_visitas
0             3               1
1             4               1
2             8               3
3            11               1
4            14               1
...         ...             ...
1210       3872               1
1211       3873               1
1212       3882               1
1213       3886               1
1214       3899               1

[1215 rows x 2 columns]


### Calcular tiempo de infeccion (dias) al ingreso. Tiempo medio en el caso de reinfecciones

In [26]:
# Calculo de dias desde desde la última infección hasta el ingreso

tbl_visitas_ip['fecha_ingreso_urgencias'] = pd.to_datetime(tbl_visitas_ip['fecha_ingreso_urgencias'])
ultima_infeccion = tbl_visitas_ip.groupby('person_id')['fecha_infeccion'].max().reset_index(name= 'ultima_fecha')
ultima_infeccion['ultima_fecha'] = pd.to_datetime(ultima_infeccion['ultima_fecha'])
fecha_ingreso_unica = tbl_visitas_ip[['person_id', 'fecha_ingreso_urgencias']].drop_duplicates()
ultima_infeccion_df = ultima_infeccion.merge(fecha_ingreso_unica, on='person_id')
ultima_infeccion_df['tiempo_ultima'] = (ultima_infeccion_df['fecha_ingreso_urgencias'] - ultima_infeccion_df['ultima_fecha']).dt.days

In [37]:
# merge infecciones previas

tbl_infecciones_complete = tbl_infecciones_previas_microorganismos.merge(num_visitas, on= ['person_id'], how= 'left')
tbl_infecciones_complete = tbl_infecciones_complete.merge(ultima_infeccion_df, on= ['person_id'], how= 'left')
tbl_infecciones_complete = tbl_infecciones_complete.drop(columns=['fecha_ingreso_urgencias_y'])
tbl_infecciones_complete.head(5)

,person_id,fecha_ingreso_urgencias_x,Enterococcus,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Staphylococcus aureus,Streptococcus pneumoniae,_Enterobacteria,_Fungi,_Other bacteria,_Virus,numero_visitas,ultima_fecha,tiempo_ultima
0,3,2021-06-04,1,0,0,0,0,0,0,0,0,0,1,2021-05-08,27
1,4,2021-06-22,1,0,0,0,0,0,0,0,0,0,1,2021-05-08,45
2,8,2020-04-13,0,0,0,0,0,0,0,1,1,1,3,2019-10-24,172
3,11,2020-03-31,0,0,0,0,0,0,0,0,1,0,1,2019-11-05,147
4,14,2021-02-24,0,1,0,0,0,0,0,0,1,0,1,2020-10-09,138


In [76]:
tbl_infecciones_complete

,person_id,fecha_ingreso_urgencias_x,Enterococcus,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Staphylococcus aureus,Streptococcus pneumoniae,_Enterobacteria,_Fungi,_Other bacteria,_Virus,numero_visitas,ultima_fecha,tiempo_ultima
0,3,2021-06-04,1,0,0,0,0,0,0,0,0,0,1,2021-05-08,27
1,4,2021-06-22,1,0,0,0,0,0,0,0,0,0,1,2021-05-08,45
2,8,2020-04-13,0,0,0,0,0,0,0,1,1,1,3,2019-10-24,172
3,11,2020-03-31,0,0,0,0,0,0,0,0,1,0,1,2019-11-05,147
4,14,2021-02-24,0,1,0,0,0,0,0,0,1,0,1,2020-10-09,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,3872,2023-11-24,0,0,0,4,0,0,0,0,0,0,1,2023-10-19,36
1157,3873,2023-02-11,0,2,0,0,0,0,0,0,0,0,1,2022-10-22,112
1158,3882,2023-05-05,0,4,0,0,0,0,0,0,0,0,1,2023-03-14,52
1159,3886,2023-01-30,0,2,0,0,0,0,0,0,0,0,1,2023-01-13,17


# Otros_cultivos_en_urgencias (sin resistencia)

In [78]:
tbl_otros_cultivos_en_urgencias = copy.deepcopy(dataframes['tbl_otros_cultivos_en_urgencias'])
tbl_otros_cultivos_en_urgencias["otro_cult_microorganismo"] = tbl_otros_cultivos_en_urgencias["microorganismo_otros_cult"].fillna("0").astype(int).astype(str).map(lambda x: organism_codes_map.get(x, "NEGATIVE"))
tbl_otros_cultivos_en_urgencias = tbl_otros_cultivos_en_urgencias.drop(columns=["microorganismo_otros_cult", "id_otros_cultivos", "fecha_otros_cultivos", "bmr_etiologia_otros", "fenotipo_resistencia_otros"])
tbl_otros_cultivos_en_urgencias["tipo_cultivo"] = tbl_otros_cultivos_en_urgencias["tipo_cultivo"].fillna(0)
tbl_otros_cultivos_en_urgencias = tbl_otros_cultivos_en_urgencias.dropna().drop_duplicates()
tbl_otros_cultivos_en_urgencias["dummy"] = 1
tbl_otros_cultivos_en_urgencias_pivoted = tbl_otros_cultivos_en_urgencias.pivot_table(
    index=["person_id", "fecha_ingreso_urgencias"],  
    columns="otro_cult_microorganismo",
    values="dummy",
    aggfunc="sum",
    fill_value=0).reset_index()
tbl_otros_cultivos_en_urgencias_pivoted = tbl_otros_cultivos_en_urgencias_pivoted.rename(columns=lambda x: f"otros_cult_{x}" if x not in ["person_id", "fecha_ingreso_urgencias"] else x)
tbl_otros_cultivos_en_urgencias_pivoted

otro_cult_microorganismo,person_id,fecha_ingreso_urgencias,otros_cult_Enterococcus,otros_cult_Escherichia coli,otros_cult_Klebsiella pneumoniae,otros_cult_NEGATIVE,otros_cult_Pseudomonas aeruginosa,otros_cult_Staphylococcus aureus,otros_cult_Streptococcus pneumoniae,otros_cult__Enterobacteria,otros_cult__Fungi,otros_cult__Other bacteria,otros_cult__Virus
0,1,2021-04-22,0,0,0,1,0,0,0,0,0,0,0
1,2,2021-05-07,0,0,0,1,0,0,0,0,0,0,0
2,3,2021-06-04,0,0,0,1,0,0,0,0,0,0,0
3,4,2021-06-22,0,0,0,1,0,0,0,0,0,0,0
4,5,2021-03-24,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3908,3909,2023-01-18,0,1,0,0,0,0,0,0,0,0,0
3909,3910,2023-04-23,0,0,0,1,0,0,0,0,0,0,0
3910,3911,2023-04-10,0,0,1,0,0,0,0,0,0,0,0
3911,3912,2023-06-19,0,0,0,1,0,0,0,0,0,0,0


# Merge 

Combinación del dataset hasta este punto.

In [85]:
df_merged = df_pacientes.merge(tbl_comorbilidad, on = ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_factores_riesgo_bmr, on = ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_sepsis, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_signos, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_sintomas_complete, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_otros_cultivos_en_urgencias_pivoted, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
orig_cols = df_merged.columns.tolist()
df_merged = df_merged.merge(tbl_infecciones_complete, on= ['person_id'], how= 'left')
df_merged = df_merged.merge(colo_prev_pivoted, on = ['person_id'], how= 'left')
new_cols = [c for c in df_merged.columns if c not in orig_cols]
df_merged[new_cols] = df_merged[new_cols].fillna(0)
df_merged = pd.merge(df_merged, hemo_urg_pivoted[["person_id", "resultado_hemo"]], on= ['person_id'], how= 'left')


In [86]:
# Create a column that sums all microorganism events
for org in set(organism_codes_map.values()):
    cols = [col for col in df_merged.columns if org in col]
    # Option A: if you want sum of counts (if columns are counts)
    df_merged[f"{org}_total"] = df_merged[cols].sum(axis=1)

In [88]:
df_expanded = df_merged.explode("resultado_hemo").reset_index(drop=True)
counts = df_expanded["person_id"].value_counts()
df_expanded["weight"] = df_expanded["person_id"].apply(lambda x: 1 / counts[x])
df_expanded["co_infection"] = np.where(df_expanded["weight"] < 1, 1, 0)
df_expanded.to_csv(os.path.join(save_location, "df_merged_full.csv"), index=False)

Sub-table for ethiology prediction (only previous events like infections and colonizations)

In [28]:
df_hemo_merged = pd.merge(df_pacientes, hemo_urg_pivoted[["person_id", "resultado_hemo"]], on= ['person_id'], how= 'left')

orig_cols = df_hemo_merged.columns.tolist()
df_hemo_merged = df_hemo_merged.merge(tbl_infecciones_complete, on= ['person_id'], how= 'left')
df_hemo_merged = df_hemo_merged.merge(colo_prev_pivoted, on = ['person_id'], how= 'left')
new_cols = [c for c in df_hemo_merged.columns if c not in orig_cols]
df_hemo_merged[new_cols] = df_hemo_merged[new_cols].fillna(0) # Not all patients have previous infections/colonizations

df_hemo_merged = df_hemo_merged.merge(tbl_sepsis, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')

In [30]:
df_expanded = df_hemo_merged.explode("resultado_hemo").reset_index(drop=True)
counts = df_expanded["person_id"].value_counts()
df_expanded["weight"] = df_expanded["person_id"].apply(lambda x: 1 / counts[x])
df_expanded["co_infection"] = np.where(df_expanded["weight"] < 1, 1, 0)

In [31]:
df_expanded.to_csv(os.path.join(save_location, "df_hemo_merged_v2.csv"), index=False)

# Specifically process certain columns

In [98]:
#df_merged["numero_visitas"] = df_merged["numero_visitas"].fillna(0)
df_merged["mujer_gestante"] = df_merged["mujer_gestante"].replace("False", 0.0).astype(int)
# Process columns related with qsofa
qsofa_components = ["taquipnea", "estado_mental_alterado", "hipotension"]
df_merged["qsofa"] = df_merged["qsofa"].where(df_merged[qsofa_components].notna().all(axis=1), np.nan)

In [99]:
recode_dictlist = [{'value': '1', 'name': ' fiebre'},
 {'value': '2', 'name': ' tos'},
 {'value': '3', 'name': ' dificultad para respirar'},
 {'value': '4', 'name': ' dolor costal'},
 {'value': '5', 'name': ' disuria'},
 {'value': '6', 'name': ' síndrome de disuria - polaquiuria'},
 {'value': '7', 'name': ' tenesmo de ano y/o recto'},
 {'value': '8', 'name': ' dolor en el ángulo renal'},
 {'value': '9', 'name': ' náuseas'},
 {'value': '10', 'name': ' vómitos'},
 {'value': '11', 'name': ' dolor abdominal'},
 {'value': '12', 'name': ' diarrea'},
 {'value': '13', 'name': ' lesión de la piel'},
 {'value': '14', 'name': ' lesión de mucosa'},
 {'value': '15', 'name': ' cefalea'},
 {'value': '16', 'name': ' dolor articular'},
 ]
recode_dict = {d["value"]: d["name"].strip().replace(" ", "_") for d in recode_dictlist}
df_merged = df_merged.drop(["sintoma_nan", "sintoma_nan_categorico"], axis=1)
df_merged = df_merged.rename(columns={
    f"sintoma_{key}.0": f"sintoma_{val}" for key, val in recode_dict.items()
})
df_merged = df_merged.rename(columns={
    f"sintoma_{key}.0_categorico": f"sintoma_{val}_categorico" for key, val in recode_dict.items()
})

In [103]:
df_merged.to_csv(os.path.join(save_location, "processed_df.csv"), index=False)

# Create html EDA report

In [ ]:
df_merged = pd.read_csv(os.path.join(save_location, "df_merged.csv"))

In [ ]:
profile = ProfileReport(df_merged, title="MePRAM EDA report")
profile.to_notebook_iframe()
profile.to_file(os.path.join(save_location, "df_merge_report.html"))

# Plot Missing Values Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def insert_linebreak(string, lengLabel=10):
    return '\n'.join(string[i:i+lengLabel] for i in range(0, len(string), lengLabel))
merged_df = pd.read_csv(os.path.join(save_location, "df_merged.csv"))
prev_num = 0

def plot_missing_columns(subset_df, title=f"Missing Data Matrix for columns 0 to 171"):
    missing_df = subset_df.isna()

    missing_df.columns = subset_df.columns
    missing_percent = missing_df.mean() * 100
    columns_with_percent = [
        f"$\\bf{{{col}}}$ ({missing_percent[col]:.2f}%)" if missing_percent[col] > 30 else f"{col} ({missing_percent[col]:.2f}%)"
        for col in subset_df.columns
    ]
    plt.figure(figsize=(24, 6))
    ax = sns.heatmap(missing_df, vmin=0, vmax=1, cbar=False,
                xticklabels=columns_with_percent)
    ax.tick_params(axis='x', which='minor', length=40)
    plt.title(title)
    plt.xlabel("Columns (% Missing)")
    plt.ylabel("Rows", rotation=90)
    plt.xticks(rotation=90)
    plt.show()


plot_missing_columns(merged_df)

missing_df = merged_df.isna()
missing_df.columns = merged_df.columns
missing_percent = missing_df.mean() * 100
print([idx for idx,x in enumerate(missing_percent) if x > 30])
dangerous_df = merged_df.iloc[:, [idx for idx,x in enumerate(missing_percent) if x > 30]]
print(dangerous_df)
plot_missing_columns(dangerous_df, f"Missing Data Matrix of {len(dangerous_df.columns)} columns with > 30% NAs")

# PCA

- Scaled data using MinMaxScaler()

Plot PCA 2D and 3D

In [ ]:
TARGET_VARIABLE = "sepsis"

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

target = df_merged[TARGET_VARIABLE]
data = df_merged.drop(columns= [TARGET_VARIABLE])

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled_data)

explained_variance = pca.explained_variance_ratio_
print(f"Varianza explicada por cada componente: {explained_variance}")
print(f"Varianza total explicada: {sum(explained_variance)}")

plt.figure(figsize=(8, 6))
plt.scatter(pca_data[:, 0], pca_data[:, 1], c=target, cmap='viridis', alpha=0.7)
plt.title("PCA 2D")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid()
plt.show()


In [ ]:
import plotly.express as px 

pca_3d = PCA(n_components=3)
pca_data_3d = pca_3d.fit_transform(scaled_data)

pca_df = pd.DataFrame(pca_data_3d, columns=["PC1", "PC2" ,"PC3"])
pca_df[TARGET_VARIABLE] = target

fig = px.scatter_3d(
    pca_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color = TARGET_VARIABLE,
    title = "PCA - 3D Visualization",
    labels= {TARGET_VARIABLE},
    color_continuous_scale="Viridis", 
    opacity=0.7  
)
fig.update_traces(marker=dict(size=5))  
fig.update_layout(scene=dict(
    xaxis_title="PC 1",
    yaxis_title="PC 2",
    zaxis_title="PC 3"
))
fig.show()

In [53]:
dataframes.keys()

dict_keys(['tbl_master', 'tbl_paciente', 'tbl_comorbilidad', 'tbl_factores_riesgo_bmr', 'tbl_sintomas', 'tbl_signos', 'tbl_sepsis', 'tbl_infecciones_previas', 'tbl_colonizaciones_previas', 'tbl_tratamiento_antibiotico_previo', 'tbl_tratamiento_empirico', 'tbl_hemocultivo_de_urgencias', 'tbl_otros_cultivos_en_urgencias', 'tbl_codes2names', 'tbl_personid2center', 'tbl_microorganismos'])

In [ ]:
processed_df_copy = df_merged

target_copy = processed_df_copy[TARGET_VARIABLE]
scaler = MinMaxScaler()
X_preprocessed = pd.DataFrame(scaler.fit_transform(processed_df_copy.drop(columns=[TARGET_VARIABLE])))
X_preprocessed[TARGET_VARIABLE] = target_copy

target_palette = {0: "blue", 1: "red"}
row_colors = X_preprocessed[TARGET_VARIABLE].map(target_palette)
X_preprocessed = X_preprocessed.dropna() 
sns.clustermap(
    X_preprocessed.drop(columns=[TARGET_VARIABLE]), 
    cmap="coolwarm",
    row_colors=row_colors,
    figsize=(30, 60),
    annot=False,
    col_cluster=False
)

plt.title("Heatmap con Clustering y Anotación por Target", pad=100)
plt.show()

# PLOT CORRELATION GRID (PAIRGRID)

In [ ]:
sintomas_columns = [col for col in sintom_sign_sepsis.columns if "sintoma" in col]
binary_columns = [col for col in sintom_sign_sepsis.columns if sintom_sign_sepsis[col].dropna().value_counts().index.isin([0, 1]).all() and col not in sintomas_columns]
numeric_columns = [col for col in sintom_sign_sepsis.columns if sintom_sign_sepsis[col].dtype in ['int64', 'float64'] and col not in binary_columns and col not in sintomas_columns]
print(binary_columns)
print(numeric_columns)
print(sintomas_columns)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

pg = sns.PairGrid(sintom_sign_sepsis[binary_columns], hue="sepsis", palette="Set2")
pg.map_diag(sns.histplot)
pg.map_offdiag(sns.barplot)
pg.add_legend()
plt.show()

In [ ]:
sintomas_columns_group1 = sintomas_columns[0:int(len(sintomas_columns)/2)]
sintomas_columns_group1 = list(set(sintomas_columns_group1 + ["sepsis"]))
sns.color_palette("tab10")
pg = sns.PairGrid(sintom_sign_sepsis[sintomas_columns_group1], hue="sepsis", palette="Set2")
pg.map_diag(sns.histplot)
pg.map_offdiag(sns.scatterplot, alpha=0.6)
pg.add_legend()
plt.show()

In [ ]:
sintomas_columns_group2 = sintomas_columns[int(len(sintomas_columns)/2):-1]
sintomas_columns_group2 = list(set(sintomas_columns_group2 + ["sepsis"]))
pg = sns.PairGrid(sintom_sign_sepsis[sintomas_columns_group2], hue="sepsis", palette="Set2")
pg.map_diag(sns.histplot, alpha=0.5)
pg.map_offdiag(sns.scatterplot, alpha=0.6)
pg.add_legend()
plt.show()

In [ ]:
numeric_columns = list(set(numeric_columns + ["sepsis"]))
pg = sns.PairGrid(sintom_sign_sepsis[numeric_columns].drop("person_id", axis=1), hue="sepsis", palette="Set2")
pg.map_diag(sns.kdeplot, alpha=0.8)
pg.map_offdiag(sns.scatterplot, alpha=0.6)
pg.add_legend()
sns.color_palette("tab10")
plt.show()